In [2]:
import torch
import torch.nn as nn
import torch.optim as optim


train_loader = 1

class MCDropoutAutoencoder(nn.Module):
    def __init__(self, input_dim=32, hidden_dim1=16, hidden_dim2=8, dropout_prob=0.2):
        super(MCDropoutAutoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.ReLU(),
            nn.Dropout(dropout_prob),  # Monte Carlo Dropout in encoder layer 1
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Dropout(dropout_prob)   # Monte Carlo Dropout in encoder layer 2
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim2, hidden_dim1),
            nn.ReLU(),
            nn.Dropout(dropout_prob),  # Monte Carlo Dropout in decoder layer 1
            nn.Linear(hidden_dim1, input_dim),
            nn.Sigmoid()  # Assuming input data is normalized between 0 and 1
        )

    def forward(self, x):
        # Encode and then decode the input
        z = self.encoder(x)
        return self.decoder(z)

# Hyperparameters
input_dim = 32
hidden_dim1 = 16
hidden_dim2 = 8
dropout_prob = 0.2
num_epochs = 20
batch_size = 64
learning_rate = 1e-3

# Instantiate model, optimizer, and loss function
model = MCDropoutAutoencoder(input_dim, hidden_dim1, hidden_dim2, dropout_prob)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, data in enumerate(train_loader):
        # Flatten input if necessary
        data = data.view(-1, input_dim)
        
        optimizer.zero_grad()
        
        # Forward pass with Monte Carlo Dropout
        output = model(data)
        
        # Compute reconstruction loss
        loss = loss_fn(output, data)
        
        # Backpropagation and optimization
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

# Function to perform Monte Carlo sampling during inference
def mc_inference(model, data, num_samples=50):
    model.eval()
    predictions = []
    with torch.no_grad():
        for _ in range(num_samples):
            output = model(data)
            predictions.append(output.unsqueeze(0))
    # Stack predictions and calculate mean and std deviation
    predictions = torch.cat(predictions, dim=0)
    mean_prediction = predictions.mean(dim=0)
    std_deviation = predictions.std(dim=0)
    return mean_prediction, std_deviation

# Example inference
# Assuming `test_data` is a batch of test samples with shape [batch_size, input_dim]
test_data = torch.randn(batch_size, input_dim)  # replace with actual test data
mean_pred, uncertainty = mc_inference(model, test_data, num_samples=50)

print("Mean prediction shape:", mean_pred.shape)
print("Uncertainty shape:", uncertainty.shape)


NameError: name 'train_loader' is not defined